In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import glob
import lightgbm as lgb

In [3]:
files = glob.glob("train/train/*.csv")
data_list = []
for file in files:
    data_list.append(pd.read_csv(file, index_col=0))
train = pd.concat(data_list)

test = pd.read_csv('test.csv')

C:\Users\reona\AppData\Local\Temp\ipykernel_32764\709303930.py:4: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  data_list.append(pd.read_csv(file, index_col=0))
C:\Users\reona\AppData\Local\Temp\ipykernel_32764\709303930.py:4: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  data_list.append(pd.read_csv(file, index_col=0))
C:\Users\reona\AppData\Local\Temp\ipykernel_32764\709303930.py:4: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  data_list.append(pd.read_csv(file, index_col=0))
C:\Users\reona\AppData\Local\Temp\ipykernel_32764\709303930.py:4: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  data_list.append(pd.read_csv(file, index_col=0))


In [4]:
train.head()

,種類,地域,市区町村コード,都道府県名,市区町村名,地区名,最寄駅：名称,最寄駅：距離（分）,間取り,面積（㎡）,...,前面道路：方位,前面道路：種類,前面道路：幅員（ｍ）,都市計画,建ぺい率（％）,容積率（％）,取引時点,改装,取引の事情等,取引価格（総額）_log
ID,,,,,,,,,,,,,,,,,,,,,
1004555,中古マンション等,NaN,1101,北海道,札幌市中央区,南２条西,西１８丁目,5,３ＬＤＫ,70,...,NaN,NaN,NaN,近隣商業地域,80.0,300.0,2009年第2四半期,改装済,NaN,6.982271
1051274,中古マンション等,NaN,1106,北海道,札幌市南区,川沿１７条,真駒内,30分?60分,４ＬＤＫ,90,...,NaN,NaN,NaN,第１種住居地域,60.0,200.0,2014年第3四半期,改装済,NaN,6.968483
1008483,中古マンション等,NaN,1101,北海道,札幌市中央区,南１４条西,西１８丁目,29,４ＬＤＫ,95,...,NaN,NaN,NaN,近隣商業地域,80.0,300.0,2013年第3四半期,改装済,NaN,7.361728
1075401,中古マンション等,NaN,1110,北海道,札幌市清田区,清田２条,福住,30分?60分,４ＬＤＫ,80,...,NaN,NaN,NaN,第１種住居地域,60.0,200.0,2013年第2四半期,未改装,NaN,6.991226
1090633,中古マンション等,NaN,1110,北海道,札幌市清田区,清田１条,福住,30分?60分,１ＬＤＫ,35,...,NaN,NaN,NaN,近隣商業地域,80.0,200.0,2008年第4四半期,未改装,NaN,6.579784


In [5]:
test.head()

,ID,種類,地域,市区町村コード,都道府県名,市区町村名,地区名,最寄駅：名称,最寄駅：距離（分）,間取り,...,今後の利用目的,前面道路：方位,前面道路：種類,前面道路：幅員（ｍ）,都市計画,建ぺい率（％）,容積率（％）,取引時点,改装,取引の事情等
0,1000077,中古マンション等,NaN,1101,北海道,札幌市中央区,大通西,西１１丁目,1,３ＬＤＫ,...,その他,NaN,NaN,NaN,商業地域,80.0,600.0,2022年第2四半期,未改装,NaN
1,1000081,中古マンション等,NaN,1101,北海道,札幌市中央区,大通西,西１１丁目,3,２ＬＤＫ,...,住宅,NaN,NaN,NaN,商業地域,80.0,600.0,2022年第2四半期,未改装,NaN
2,1000128,中古マンション等,NaN,1101,北海道,札幌市中央区,大通西,西１８丁目,2,２ＬＤＫ,...,住宅,NaN,NaN,NaN,商業地域,80.0,400.0,2022年第3四半期,未改装,NaN
3,1000129,中古マンション等,NaN,1101,北海道,札幌市中央区,大通西,西１８丁目,2,１ＤＫ,...,住宅,NaN,NaN,NaN,商業地域,80.0,400.0,2022年第3四半期,未改装,NaN
4,1000130,中古マンション等,NaN,1101,北海道,札幌市中央区,大通西,西１８丁目,1,３ＬＤＫ,...,住宅,NaN,NaN,NaN,商業地域,80.0,400.0,2022年第3四半期,未改装,NaN


In [6]:
train.shape

(765412, 27)

In [7]:
test.shape

(19301, 27)

In [8]:
train.isnull().sum()

種類                   0
地域              765412
市区町村コード              0
都道府県名                0
市区町村名                0
地区名                671
最寄駅：名称            2789
最寄駅：距離（分）        23215
間取り              28323
面積（㎡）                0
土地の形状           765412
間口              765412
延床面積（㎡）         765412
建築年              22366
建物の構造            21575
用途               82264
今後の利用目的         367658
前面道路：方位         765412
前面道路：種類         765412
前面道路：幅員（ｍ）      765412
都市計画             20452
建ぺい率（％）          25009
容積率（％）           25009
取引時点                 0
改装               74204
取引の事情等          746142
取引価格（総額）_log         0
dtype: int64

In [30]:
test.isnull().sum()

ID                0
種類                0
地域            19301
市区町村コード           0
都道府県名             0
市区町村名             0
地区名               1
最寄駅：名称           11
最寄駅：距離（分）        11
間取り            1197
面積（㎡）             0
土地の形状         19301
間口            19301
延床面積（㎡）       19301
建築年             590
建物の構造           977
用途             4657
今後の利用目的         810
前面道路：方位       19301
前面道路：種類       19301
前面道路：幅員（ｍ）    19301
都市計画            246
建ぺい率（％）         348
容積率（％）          348
取引時点              0
改装             3093
取引の事情等        19130
dtype: int64

In [9]:
train.drop(columns=["地域","間口","延床面積（㎡）","前面道路：方位","前面道路：種類","前面道路：幅員（ｍ）","土地の形状","取引の事情等"],axis=1,inplace=True)
test.drop(columns=["ID","地域","間口","延床面積（㎡）","前面道路：方位","前面道路：種類","前面道路：幅員（ｍ）","土地の形状","取引の事情等"],axis=1,inplace=True)

In [10]:
print(train.shape)
print(test.shape)

(765412, 19)
(19301, 18)


In [11]:
train.isnull().sum()

種類                   0
市区町村コード              0
都道府県名                0
市区町村名                0
地区名                671
最寄駅：名称            2789
最寄駅：距離（分）        23215
間取り              28323
面積（㎡）                0
建築年              22366
建物の構造            21575
用途               82264
今後の利用目的         367658
都市計画             20452
建ぺい率（％）          25009
容積率（％）           25009
取引時点                 0
改装               74204
取引価格（総額）_log         0
dtype: int64

In [12]:
train.to_csv('train_1.csv', index=False)

In [13]:
test.to_csv('test_1.csv', index=False)

In [ ]:
// 前処理途中のファイルはtrain_1.csv test_1.csvに保存